In [2]:
import pandas as pd

df = pd.read_excel('merged_files.xlsx')

In [3]:
import datetime

def extract_hour(dt):
    return dt.hour

df['시간'] = df['주문시간'].apply(extract_hour)


In [4]:
# 일자와 시간으로 그룹화
grouped = df.groupby([df['주문시간'].dt.date, '시간'])

# 상품 판매량을 각 상품마다 더해서 넣어줌
sales = grouped['상 품 명'].value_counts().unstack(fill_value=0)

# Resetting index to make '일자' and '시간' as columns
sales = sales.reset_index()

# 주문시간이라고 되어있는 열을 일자로 바꾸어줌 (시간은 따로 빼서)
sales.rename(columns={'주문시간': '일자'}, inplace=True)

In [5]:
sales.columns

Index(['일자', '시간', '.', '101Row(s)', '103Row(s)', '105Row(s)', '109Row(s)',
       '110Row(s)', '111Row(s)', '114Row(s)',
       ...
       '할인', '핫초코', '햄.치즈샌드위치', '행사) 아메리카노', '행사) 아이스아메리카노', '홍차라떼', '화이트모카',
       '화이트민트초코라떼', '휘핑크림추가', '히비스커스'],
      dtype='object', name='상 품 명', length=292)

In [6]:
import re

# '(숫자)Row(s)' 로 들어가게 된 열 제거를 위해 정규표현식 사용!
pattern = r'\d+Row\(s\)'

# 패턴에 맞춰 열 찾기
remove_pattern = [col for col in sales.columns if re.search(pattern, col)]
sales_cleaned_pattern = sales.drop(columns=remove_pattern)

sales_cleaned_pattern.head()

상 품 명,일자,시간,.,HOT아메리카노 할인,ICE아메리카노 할인,NO WHIP,가은블랜딩원두100g,가은블랜딩원두200g,고구마라떼,골드메달 (애플),...,할인,핫초코,햄.치즈샌드위치,행사) 아메리카노,행사) 아이스아메리카노,홍차라떼,화이트모카,화이트민트초코라떼,휘핑크림추가,히비스커스
0,2022-03-02,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2022-03-02,9,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2022-03-02,10,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2022-03-02,11,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0
4,2022-03-02,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [7]:
sales_cleaned_pattern.drop(columns='.', inplace=True)

In [8]:
# 각 행의 주문 합계를 더함 (숫자형으로 맞춰줘야해서 일자와 시간은 빼고 더함)
sales_cleaned_pattern['합계'] = sales_cleaned_pattern.drop(['일자', '시간'], axis=1).sum(axis=1)
sales_cleaned_pattern.head()

상 품 명,일자,시간,HOT아메리카노 할인,ICE아메리카노 할인,NO WHIP,가은블랜딩원두100g,가은블랜딩원두200g,고구마라떼,골드메달 (애플),골드메달(스파클링),...,핫초코,햄.치즈샌드위치,행사) 아메리카노,행사) 아이스아메리카노,홍차라떼,화이트모카,화이트민트초코라떼,휘핑크림추가,히비스커스,합계
0,2022-03-02,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
1,2022-03-02,9,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,21
2,2022-03-02,10,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,21
3,2022-03-02,11,0,0,0,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,62
4,2022-03-02,12,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,47


In [9]:
print(list(sales_cleaned_pattern.columns))

['일자', '시간', 'HOT아메리카노 할인', 'ICE아메리카노 할인', 'NO WHIP', '가은블랜딩원두100g', '가은블랜딩원두200g', '고구마라떼', '골드메달    (애플)', '골드메달(스파클링)', '녹차라떼', '덜달게', '딸기라떼', '딸기스무디', '딸기연유라떼 14oz', '레모니카노', '레몬글라스', '레몬에이드', '레몬차', '레몬차 ICE', '많이(얼음/물/우유)', '망고스무디', '망고에이드', '민트초코', '바닐라라떼', '복숭아아이스티', '분다버그      (라임, 자몽)', '블루베리라떼', '블루베리스무디', '사이즈업 중간잔 (AME)', '사이즈업 큰잔', '상품권', '생강차', '생강차 ICE', '샷추가', '석류아이스티', '시럽추가', '아메리카노', '아이리쉬라떼', '아이스 레몬글라스', '아이스 얼그레이홍차', '아이스 잉글리쉬 블랙퍼스트', '아이스 캐모마일', '아이스 페퍼민트', '아이스 히비스커스', '아이스고구마라떼', '아이스녹차라떼', '아이스딸기라떼', '아이스레모니카노', '아이스민트초코(휘핑)', '아이스바닐라라떼', '아이스블루베리라떼', '아이스아메리카노', '아이스아이리쉬라떼', '아이스연유라떼', '아이스오곡라떼', '아이스초코(휘핑)', '아이스카라멜마끼아또', '아이스카페라떼', '아이스카페모카', '아이스토피넛라떼', '아이스토피넛모카', '아이스홍차라떼', '아이스화이트모카', '아이스화이트민트초코라떼', '아포가토', '얼그레이홍차', '얼음컵', '에스프레소', '연유라떼', '오곡라떼', '오렌지에이드', '우유', '유자스무디', '유자에이드', '유자차', '유자차 ICE', '잉글리쉬 블랙퍼스트', '자몽에이드', '자몽차', '자몽차 ICE', '적게(얼음/물/우유)', '젤라또', '직무박람회할인', '천혜향스무디', '천혜향에이드', '천혜향차', '천혜향차 ICE', '청포도에이드', '카라멜마끼아또', '카야샌드위치', '카페라떼', '

In [10]:
sales_cleaned_pattern.to_excel("check.xlsx", index=False)

In [11]:
sales_cleaned_pattern['아이스아메리카노_sum'] = sales_cleaned_pattern['아이스아메리카노'] + sales_cleaned_pattern['ICE아메리카노 할인'] + sales_cleaned_pattern['행사) 아이스아메리카노']

In [12]:
sales_cleaned_pattern['아메리카노_sum'] = sales_cleaned_pattern['아메리카노'] + sales_cleaned_pattern['HOT아메리카노 할인'] + sales_cleaned_pattern['행사) 아메리카노']
sales_cleaned_pattern.head(87)   # 데이터 확인

상 품 명,일자,시간,HOT아메리카노 할인,ICE아메리카노 할인,NO WHIP,가은블랜딩원두100g,가은블랜딩원두200g,고구마라떼,골드메달 (애플),골드메달(스파클링),...,행사) 아메리카노,행사) 아이스아메리카노,홍차라떼,화이트모카,화이트민트초코라떼,휘핑크림추가,히비스커스,합계,아이스아메리카노_sum,아메리카노_sum
0,2022-03-02,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,20,3,2
1,2022-03-02,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,21,1,3
2,2022-03-02,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,21,3,2
3,2022-03-02,11,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,62,4,0
4,2022-03-02,12,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,47,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,2022-03-15,11,0,0,0,0,0,0,0,0,...,3,12,0,1,0,0,0,43,12,3
83,2022-03-15,12,0,0,0,0,0,1,0,0,...,4,12,2,0,1,0,0,89,12,4
84,2022-03-15,13,0,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,48,2,2
85,2022-03-15,14,0,0,0,0,0,0,0,0,...,2,4,0,0,0,0,0,49,4,2


In [13]:
# 기존 6개(3개+3개)의 column 삭제
sales_cleaned_pattern = sales_cleaned_pattern.drop(['아이스아메리카노', 'ICE아메리카노 할인', '행사) 아이스아메리카노', '아메리카노', 'HOT아메리카노 할인', '행사) 아메리카노'], axis=1)
sales_cleaned_pattern

상 품 명,일자,시간,NO WHIP,가은블랜딩원두100g,가은블랜딩원두200g,고구마라떼,골드메달 (애플),골드메달(스파클링),녹차라떼,덜달게,...,핫초코,햄.치즈샌드위치,홍차라떼,화이트모카,화이트민트초코라떼,휘핑크림추가,히비스커스,합계,아이스아메리카노_sum,아메리카노_sum
0,2022-03-02,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,20,3,2
1,2022-03-02,9,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,21,1,3
2,2022-03-02,10,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,21,3,2
3,2022-03-02,11,0,0,0,1,0,0,1,0,...,1,0,0,1,0,0,0,62,4,0
4,2022-03-02,12,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,47,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906,2022-12-30,9,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,23,3,1
1907,2022-12-30,10,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,20,2,2
1908,2022-12-30,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
1909,2022-12-30,12,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,22,5,3


In [14]:
# column 이름 변경
sales_cleaned_pattern.rename(columns={'아이스아메리카노_sum': '아이스아메리카노'}, inplace=True)
sales_cleaned_pattern.rename(columns={'아메리카노_sum': '아메리카노'}, inplace=True)

In [15]:
sales_cleaned_pattern.info()   # 데이터 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1911 entries, 0 to 1910
Columns: 114 entries, 일자 to 아메리카노
dtypes: int64(113), object(1)
memory usage: 1.7+ MB


In [16]:
list(sales_cleaned_pattern.columns)   # 칼럼 확인

['일자',
 '시간',
 'NO WHIP',
 '가은블랜딩원두100g',
 '가은블랜딩원두200g',
 '고구마라떼',
 '골드메달    (애플)',
 '골드메달(스파클링)',
 '녹차라떼',
 '덜달게',
 '딸기라떼',
 '딸기스무디',
 '딸기연유라떼 14oz',
 '레모니카노',
 '레몬글라스',
 '레몬에이드',
 '레몬차',
 '레몬차 ICE',
 '많이(얼음/물/우유)',
 '망고스무디',
 '망고에이드',
 '민트초코',
 '바닐라라떼',
 '복숭아아이스티',
 '분다버그      (라임, 자몽)',
 '블루베리라떼',
 '블루베리스무디',
 '사이즈업 중간잔 (AME)',
 '사이즈업 큰잔',
 '상품권',
 '생강차',
 '생강차 ICE',
 '샷추가',
 '석류아이스티',
 '시럽추가',
 '아이리쉬라떼',
 '아이스 레몬글라스',
 '아이스 얼그레이홍차',
 '아이스 잉글리쉬 블랙퍼스트',
 '아이스 캐모마일',
 '아이스 페퍼민트',
 '아이스 히비스커스',
 '아이스고구마라떼',
 '아이스녹차라떼',
 '아이스딸기라떼',
 '아이스레모니카노',
 '아이스민트초코(휘핑)',
 '아이스바닐라라떼',
 '아이스블루베리라떼',
 '아이스아이리쉬라떼',
 '아이스연유라떼',
 '아이스오곡라떼',
 '아이스초코(휘핑)',
 '아이스카라멜마끼아또',
 '아이스카페라떼',
 '아이스카페모카',
 '아이스토피넛라떼',
 '아이스토피넛모카',
 '아이스홍차라떼',
 '아이스화이트모카',
 '아이스화이트민트초코라떼',
 '아포가토',
 '얼그레이홍차',
 '얼음컵',
 '에스프레소',
 '연유라떼',
 '오곡라떼',
 '오렌지에이드',
 '우유',
 '유자스무디',
 '유자에이드',
 '유자차',
 '유자차 ICE',
 '잉글리쉬 블랙퍼스트',
 '자몽에이드',
 '자몽차',
 '자몽차 ICE',
 '적게(얼음/물/우유)',
 '젤라또',
 '직무박람회할인',
 '천혜향스무디',
 '천혜향에이드',
 '천혜향차',
 '천혜향차 ICE',
 '청포도에이드

In [17]:
# 삭제할 칼럼 지정
columns_to_drop = ['일자', '시간', 'NO WHIP', '가은블랜딩원두100g', '가은블랜딩원두200g',
                   '덜달게', '많이(얼음/물/우유)', '골드메달    (애플)', '골드메달(스파클링)',
                   '분다버그      (라임, 자몽)', '사이즈업 중간잔 (AME)', '사이즈업 큰잔',
                   '상품권', '시럽추가', '얼음컵', '적게(얼음/물/우유)', '직무박람회할인',
                   '캐리어', '컵', '쿠키', '쿠폰', '쿠폰(상품권)', '텀블러, 머그할인',
                   '할인', '휘핑크림추가', '합계']

sales_cleaned_pattern.drop(columns=columns_to_drop, inplace=True)

In [18]:
sales_cleaned_pattern

상 품 명,고구마라떼,녹차라떼,딸기라떼,딸기스무디,딸기연유라떼 14oz,레모니카노,레몬글라스,레몬에이드,레몬차,레몬차 ICE,...,페퍼민트,플레인스무디,핫초코,햄.치즈샌드위치,홍차라떼,화이트모카,화이트민트초코라떼,히비스커스,아이스아메리카노,아메리카노
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,3,2
3,1,1,0,0,2,0,0,1,0,0,...,0,0,1,0,0,1,0,0,4,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906,0,0,0,0,1,0,0,0,1,0,...,1,0,1,0,0,0,0,0,3,1
1907,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,2,2
1908,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1909,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,3


In [19]:
sales_cleaned_pattern.to_csv('merge_menu_sum.csv')